In [32]:
import pandas as pd

In [33]:
df= pd.read_csv("DT.csv")

In [34]:
df

,No.,WT74_On,Edited74_On,RT-PBS_right4,AfterRTT_left4,PBSlen,RTlen,RT-PBSlen,Edit_pos,Edit_len,...,fGCcont2,fGCcont3,MFE1,MFE2,MFE3,MFE4,MFE5,DeepSpCas9_score,Measured_PE_efficiency,Fold
0,1,GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...,xxxxxxxxxxGTGGAGGGAGGTCCGGGGTGAAAATxxxxxxxxxxx...,AAAT,TCAT,11,14,25,4,1,...,50.000000,60.000000,-50.4,-25.0,-2.0,-2.3,-25.4,21.012074,9.890110,Test
1,2,GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...,xxxxxxxxGCGTGGAGGGAGGTCCGGGGTGAAxxxxxxxxxxxxxx...,TGAA,AAAT,13,11,24,4,1,...,63.636364,70.833336,-57.5,-25.7,-2.0,-2.3,-25.4,21.012074,3.436989,Test
2,3,GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...,xxxxxxxxxxxxxxxxxxAGGTCCGGGGTxxxxxxxxxxxxxxxxx...,GGGT,TGAA,3,8,11,4,1,...,75.000000,72.727270,-31.5,-22.4,0.0,-2.3,-25.4,21.012074,0.000000,Test
3,4,GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...,xxxxxxxxxxxTGGAGGGAGGTCCGGGGxxxxxxxxxxxxxxxxxx...,GGGG,GTGA,10,7,17,4,1,...,85.714290,76.470590,-50.2,-27.7,-2.0,-2.3,-25.4,21.012074,0.000000,Test
4,5,GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...,xxxxxTTGGCGTGGAGGGAGGTCCGGGGTGAAAATCATATCTATTC...,ATTC,CTGA,16,25,41,4,1,...,40.000000,51.219513,-62.0,-25.7,-2.0,-2.3,-25.4,21.012074,1.345292,Test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288788,548131,TTTATTTCTGTAATTTCATCCACTTGGATGCGGGGTTGTTTCTAAG...,xxxxxxxxxxxxxxxxxxTCCACTTGGATGCGGGGGACTTTCTAAG...,CAAT,AACT,3,30,33,15,3,...,46.666668,48.484848,-26.1,-24.6,-6.0,0.0,-18.7,16.009068,0.000000,4
288789,548132,TTTATTTCTGTAATTTCATCCACTTGGATGCGGGGTTGTTTCTAAG...,xxxxxxxxxxxxxxxxCATCCACTTGGATGCGGGGGACTTTCTAAG...,AACT,CGCA,5,34,39,15,3,...,44.117645,46.153847,-28.7,-26.0,-7.4,0.0,-18.7,16.009068,0.000000,4
288790,548134,TTTATTTCTGTAATTTCATCCACTTGGATGCGGGGTTGTTTCTAAG...,xxxxxxxxxxxxxxTTCATCCACTTGGATGCGGGGGACTTTCTAAG...,ACAA,TAAC,7,29,36,15,3,...,48.275864,47.222220,-32.3,-25.7,-6.4,0.0,-18.7,16.009068,0.000000,4
288791,548135,TTTATTTCTGTAATTTCATCCACTTGGATGCGGGGTTGTTTCTAAG...,xxxxxxxxxxxxxxxxxxxCCACTTGGATGCGGGGGACTTTCTxxx...,TTCT,AAGA,2,22,24,15,3,...,54.545456,58.333332,-24.4,-22.0,-4.3,0.0,-18.7,16.009068,0.000000,4


In [35]:
print(df['Fold'].unique())

['Test' '2' '4' '0' '1' '3']


In [36]:
print(df['Fold'].value_counts())

Fold
0       66797
1       59465
2       54496
3       48065
4       31087
Test    28883
Name: count, dtype: int64


In [37]:
df.size

9818962

In [38]:
pip install transformers torch pandas numpy


Note: you may need to restart the kernel to use updated packages.


In [39]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.add_special_tokens({'additional_special_tokens': ['[A]', '[T]', '[G]', '[C]', '[N]', '[PAD]']})


5

In [40]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
import torch.nn as nn

In [ ]:
df = pd.read_csv("DT.csv")

# Convert all numerical columns to float32
for col in df.select_dtypes(include=['float64']).columns:
    df[col] = df[col].astype(np.float32)




In [ ]:
df.size

9818962

In [53]:
class CRISPRDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get sequences
        wt_seq = self.df['WT74_On'][idx].lower()
        edited_seq = self.df['Edited74_On'][idx].lower()

        # Replace 'x' with '[PAD]' for better tokenization
        edited_seq = edited_seq.replace('x', '[PAD]')

        # Create feature dictionary
        features = {
            'PBS_len': self.df['PBSlen'][idx],
            'RT_len': self.df['RTlen'][idx],
            'edit_pos': self.df['Edit_pos'][idx],
            'edit_len': self.df['Edit_len'][idx],
            'RHA_len': self.df['RHA_len'][idx],
            'type_sub': self.df['type_sub'][idx],
            'type_ins': self.df['type_ins'][idx],
            'type_del': self.df['type_del'][idx],
            'GC_content': self.calculate_gc_content(wt_seq),
            'tm_features': torch.tensor([
                self.df['Tm1'][idx],
                self.df['Tm2'][idx],
                self.df['Tm3'][idx],
                self.df['Tm4'][idx],
            ], dtype=torch.float32),
            'mfe_features': torch.tensor([
                self.df['MFE1'][idx],
                self.df['MFE2'][idx],
                self.df['MFE3'][idx],
                self.df['MFE4'][idx],
                self.df['MFE5'][idx],
            ], dtype=torch.float32),
        }

        # Tokenize sequences
        wt_tokens = self.tokenize_sequence(wt_seq)
        edited_tokens = self.tokenize_sequence(edited_seq)

        # Combine all features
        inputs = {
            'wt_tokens': wt_tokens,
            'edited_tokens': edited_tokens,
            'features': features,
        }

        # Target
        target = torch.tensor(self.df['Measured_PE_efficiency'][idx], dtype=torch.float32)

        return inputs, target

    def tokenize_sequence(self, sequence):
        """Tokenize DNA sequence using BERT tokenizer"""
        # Split sequence into individual nucleotides
        seq_chars = " ".join(list(sequence))
        tokens = self.tokenizer(
            seq_chars,
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_tensors='pt'
        )
        return {k: v.squeeze(0) for k, v in tokens.items()}

    def calculate_gc_content(self, sequence):
        """Calculate GC content of sequence"""
        seq = sequence.lower().replace('x', '')
        if not seq:
            return 0
        gc_count = seq.count('g') + seq.count('c')
        return gc_count / len(seq)

def prepare_crispr_data(df_path, batch_size=32):
    """Prepare CRISPR data for training"""
    # Load data
    df = pd.read_csv(df_path)

    # Split data
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

    # Create tokenizer for DNA sequences
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    # Add special tokens for nucleotides
    special_tokens = {'additional_special_tokens': ['[A]', '[T]', '[G]', '[C]', '[N]', '[PAD]']}
    tokenizer.add_special_tokens(special_tokens)

    # Create datasets
    train_dataset = CRISPRDataset(train_df, tokenizer)
    val_dataset = CRISPRDataset(val_df, tokenizer)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, tokenizer
df_path = "DT.csv"
train_loader, val_loader, tokenizer = prepare_crispr_data(df_path)


In [54]:
print(df.head())

   No.                                            WT74_On  \
0    1  GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...   
1    2  GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...   
2    3  GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...   
3    4  GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...   
4    5  GTGTTTTGGCGTGGAGGGAGGTCCAGGGGTGAAAATCATATCTATT...   

                                         Edited74_On RT-PBS_right4  \
0  xxxxxxxxxxGTGGAGGGAGGTCCGGGGTGAAAATxxxxxxxxxxx...          AAAT   
1  xxxxxxxxGCGTGGAGGGAGGTCCGGGGTGAAxxxxxxxxxxxxxx...          TGAA   
2  xxxxxxxxxxxxxxxxxxAGGTCCGGGGTxxxxxxxxxxxxxxxxx...          GGGT   
3  xxxxxxxxxxxTGGAGGGAGGTCCGGGGxxxxxxxxxxxxxxxxxx...          GGGG   
4  xxxxxTTGGCGTGGAGGGAGGTCCGGGGTGAAAATCATATCTATTC...          ATTC   

  AfterRTT_left4  PBSlen  RTlen  RT-PBSlen  Edit_pos  Edit_len  ...  \
0           TCAT      11     14         25         4         1  ...   
1           AAAT      13     11         24         4         1  ...   

In [55]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from tqdm import tqdm

class IntegratedCRISPRDataset(Dataset):
    def __init__(self, df, tokenizer=None, use_bert=True, use_onehot=True, max_length=128, max_edited_length=10):
        """
        Integrated dataset that combines one-hot encoding and BERT tokenization.

        Args:
            df: DataFrame containing CRISPR data.
            tokenizer: BERT tokenizer (if use_bert=True).
            use_bert: Whether to use BERT tokenization.
            use_onehot: Whether to use one-hot encoding.
            max_length: Maximum sequence length for tokenization.
            max_edited_length: Maximum length of edits.
        """
        self.df = df.reset_index(drop=True)
        self.use_bert = use_bert
        self.use_onehot = use_onehot
        self.max_length = max_length
        self.max_edited_length = max_edited_length
        self.tokenizer = tokenizer

        if not (use_bert or use_onehot):
            raise ValueError("At least one of use_bert or use_onehot must be True")

        # Precompute max sequence length
        self.total_max_length = max(len(seq) for seq in df['WT74_On']) + max_edited_length

        # Preprocess one-hot features to improve efficiency if enabled
        if self.use_onehot:
            print("Preprocessing one-hot encodings...")
            self.onehot_features = self._preprocess_onehot_features()

    def _preprocess_onehot_features(self):
        """Preprocess all sequences with one-hot encoding method."""
        onehot_features = []

        for i in tqdm(range(len(self.df))):
            # Get sequences and convert to lower case
            dna = self.df['WT74_On'][i].lower()
            target = self.df['Edited74_On'][i].lower()

            # Determine PBS start position based on first non-'x' in target
            for ii in range(len(dna)):
                if ii < len(target) and target[ii] != 'x':
                    break

            pbs_start = ii
            pbs_end = ii + self.df['PBSlen'][i]
            rtt_start = pbs_end + 1
            edit_start = pbs_end + self.df['Edit_pos'][i]
            edit_length = self.df['Edit_len'][i]

            # Process edit types
            if self.df['type_del'][i] == 1:
                edit_end = edit_start + edit_length
                target = target[:edit_start-1] + 'd' * edit_length + target[edit_start-1:]
            elif self.df['type_ins'][i] == 1:
                edit_end = edit_start + edit_length
                dna = dna[:edit_start] + 'd' * edit_length + dna[edit_start:]
            elif self.df['type_sub'][i] == 1:
                edit_end = edit_start + edit_length

            # Find RHA end (last index with non-'x' in target)
            for iii in reversed(range(min(len(dna), len(target)))):
                if iii < len(target) and target[iii] != 'x':
                    break
            rha_end = iii

            # Add padding to sequences
            padding_dna = self.total_max_length - len(dna)
            dna_padded = dna + 'x' * padding_dna

            padding_target = self.total_max_length - len(target)
            target_padded = target + 'x' * padding_target

            # Convert sequences to one-hot encoding
            dna_onehot = self._letter_to_onehot(dna_padded)
            target_onehot = self._letter_to_onehot(target_padded)

            # Create feature masks for PBS, deletion/insertion/substitution, and RTT regions
            pbs = torch.zeros(len(target_padded))
            pbs[pbs_start:pbs_end] = 1
            pbs = pbs.long().unsqueeze(dim=1)

            del_target = torch.zeros(len(target_padded))
            ins_dna = torch.zeros(len(dna_padded))
            sub_dna = torch.zeros(len(dna_padded))

            if self.df['type_del'][i] == 1:
                del_target[edit_start-1:edit_end-1] = 1
                rtt = torch.zeros(len(target_padded))
                rtt[rtt_start-1:rha_end+1] = 1
                rtt[edit_start-1:edit_end-1] = 0
                rtt = rtt.long().unsqueeze(dim=1)
            elif self.df['type_ins'][i] == 1:
                ins_dna[edit_start:edit_end] = 1
                rtt = torch.zeros(len(target_padded))
                rtt[rtt_start-1:rha_end+1] = 1
                rtt[edit_start:edit_end] = 0
                rtt = rtt.long().unsqueeze(dim=1)
            elif self.df['type_sub'][i] == 1:
                sub_dna[edit_start-1:edit_end-1] = 1
                rtt = torch.zeros(len(target_padded))
                rtt[rtt_start-1:rha_end+1] = 1
                rtt[edit_start-1:edit_end-1] = 0
                rtt = rtt.long().unsqueeze(dim=1)

            del_target = del_target.long().unsqueeze(dim=1)
            ins_dna = ins_dna.long().unsqueeze(dim=1)
            sub_dna = sub_dna.long().unsqueeze(dim=1)

            # Combine all one-hot features into a single tensor per sequence
            result = torch.cat([dna_onehot, target_onehot, pbs, rtt, del_target, ins_dna, sub_dna], dim=1)

            onehot_features.append(result)

        return onehot_features

    def _letter_to_onehot(self, dna):
        """Convert DNA sequence to one-hot encoding."""
        dna = dna.lower()
        mapping = {'a': 0, 'c': 1, 't': 2, 'g': 3, 'x': 4, 'd': 4}
        encoded = torch.zeros((len(dna), 5), dtype=torch.long)
        for i, nucleotide in enumerate(dna):
            encoded[i, mapping.get(nucleotide, 4)] = 1
        return encoded

    def __len__(self):
        return len(self.df)

    def calculate_gc_content(self, sequence):
        """Calculate GC content of sequence."""
        seq = sequence.lower().replace('x', '')
        if not seq:
            return 0
        gc_count = seq.count('g') + seq.count('c')
        return gc_count / len(seq)

    def tokenize_sequence(self, sequence):
        """Tokenize DNA sequence using BERT tokenizer."""
        seq_chars = " ".join(list(sequence))  # Space-separate nucleotides
        tokens = self.tokenizer(
            seq_chars,
            padding='max_length',
            max_length=self.max_length,
            truncation=True,
            return_tensors='pt'
        )
        return {k: v.squeeze(0) for k, v in tokens.items()}

    def __getitem__(self, idx):
        # Retrieve raw sequences
        wt_seq = self.df.loc[idx, 'WT74_On'].lower()
        edited_seq = self.df.loc[idx, 'Edited74_On'].lower()

        # Initialize features dictionary
        features = {}

        # Add one-hot encoded features if enabled
        if self.use_onehot:
            features['onehot_encoding'] = self.onehot_features[idx]

        # Add BERT tokenized features if enabled
        if self.use_bert:
            # Clean the edited sequence for tokenization
            edited_seq_cleaned = edited_seq.replace('x', '[PAD]')
            features['wt_tokens'] = self.tokenize_sequence(wt_seq)
            features['edited_tokens'] = self.tokenize_sequence(edited_seq_cleaned)

        # Add numerical features
        numerical_features = {
            'PBS_len': self.df.loc[idx, 'PBSlen'],
            'RT_len': self.df.loc[idx, 'RTlen'],
            'edit_pos': self.df.loc[idx, 'Edit_pos'],
            'edit_len': self.df.loc[idx, 'Edit_len'],
            'RHA_len': self.df.loc[idx, 'RHA_len'],
            'type_sub': self.df.loc[idx, 'type_sub'],
            'type_ins': self.df.loc[idx, 'type_ins'],
            'type_del': self.df.loc[idx, 'type_del'],
            'GC_content': self.calculate_gc_content(wt_seq),
        }

        if 'Tm1' in self.df.columns:
            numerical_features['tm_features'] = torch.tensor([
                self.df.loc[idx, 'Tm1'],
                self.df.loc[idx, 'Tm2'],
                self.df.loc[idx, 'Tm3'],
                self.df.loc[idx, 'Tm4'],
            ], dtype=torch.float32)

        if 'MFE1' in self.df.columns:
            numerical_features['mfe_features'] = torch.tensor([
                self.df.loc[idx, 'MFE1'],
                self.df.loc[idx, 'MFE2'],
                self.df.loc[idx, 'MFE3'],
                self.df.loc[idx, 'MFE4'],
                self.df.loc[idx, 'MFE5'],
            ], dtype=torch.float32)

        features['numerical_features'] = numerical_features

        # Target variable
        target = torch.tensor(self.df.loc[idx, 'Measured_PE_efficiency'], dtype=torch.float32)

        return features, target

def prepare_integrated_crispr_data(df_path, batch_size=32, use_bert=True, use_onehot=True):
    """Prepare integrated CRISPR data for training."""
    # Load data based on file extension
    if isinstance(df_path, str):
        if df_path.endswith('.csv'):
            df = pd.read_csv(df_path)
        elif df_path.endswith('.xlsx') or df_path.endswith('.xls'):
            df = pd.read_excel(df_path)
        else:
            raise ValueError(f"Unsupported file format: {df_path}")
    else:
        df = df_path  # Assume it's already a DataFrame

    # Split data into train and validation sets
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

    tokenizer = None
    if use_bert:
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        special_tokens = {'additional_special_tokens': ['[A]', '[T]', '[G]', '[C]', '[N]', '[PAD]']}
        tokenizer.add_special_tokens(special_tokens)

    train_dataset = IntegratedCRISPRDataset(train_df, tokenizer, use_bert, use_onehot)
    val_dataset = IntegratedCRISPRDataset(val_df, tokenizer, use_bert, use_onehot)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, tokenizer


In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel

class CRISPRPredictor(nn.Module):
    def __init__(self, tokenizer, dropout_rate=0.2):
        super(CRISPRPredictor, self).__init__()

        # DNA sequence encoder using BERT
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        # Resize token embeddings to account for additional tokens
        self.bert.resize_token_embeddings(len(tokenizer))

        # Hidden dimension from BERT
        self.seq_dim = self.bert.config.hidden_size
        # Dimension for combined non-sequence features after processing
        self.feature_dim = 16

        # Embeddings for categorical features
        self.pbs_embedding = nn.Embedding(50, 8)         # PBS length embedding
        self.rt_embedding = nn.Embedding(50, 8)          # RT length embedding
        self.edit_pos_embedding = nn.Embedding(50, 8)
        self.edit_len_embedding = nn.Embedding(20, 8)
        self.rha_len_embedding = nn.Embedding(50, 8)

        # Embedding for edit type (0: sub, 1: ins, 2: del)
        self.edit_type_embedding = nn.Embedding(3, 8)

        # Linear layers for numerical features
        self.tm_features_linear = nn.Linear(4, 16)
        self.mfe_features_linear = nn.Linear(5, 16)
        self.gc_linear = nn.Linear(1, 8)

        # Combine all non-sequence features into a fixed dimension
        # Total concatenated dimension: 8*5 (from 5 embeddings) + 8 (edit type) + 16 (tm) + 16 (mfe) + 8 (gc) = 80
        self.feature_linear = nn.Linear(80, self.feature_dim)

        # Self-attention mechanism for sequence features
        self.attention = nn.MultiheadAttention(embed_dim=self.seq_dim, num_heads=8, dropout=dropout_rate)

        # Final combined processing
        self.combined_linear = nn.Sequential(
            nn.Linear(self.seq_dim + self.feature_dim, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1)
        )

        # Sigmoid activation to scale output (we later multiply by 100 for percentage)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        # Process sequence data with BERT:
        wt_output = self.bert(**inputs['wt_tokens']).last_hidden_state  # shape: [batch, seq_len, hidden_size]
        edited_output = self.bert(**inputs['edited_tokens']).last_hidden_state

        # Apply self-attention to capture context (using the same tensor for query, key, value)
        wt_attn, _ = self.attention(wt_output, wt_output, wt_output)
        edited_attn, _ = self.attention(edited_output, edited_output, edited_output)

        # Mean pooling over sequence length
        wt_pooled = torch.mean(wt_attn, dim=1)       # shape: [batch, hidden_size]
        edited_pooled = torch.mean(edited_attn, dim=1)

        # Difference between edited and wild-type to capture change
        seq_features = edited_pooled - wt_pooled      # shape: [batch, hidden_size]

        # Process numerical/categorical features.
        # NOTE: Our IntegratedCRISPRDataset returns these under 'numerical_features'
        features = inputs['numerical_features']

        # Embed categorical features (clamping values to avoid index errors)
        pbs_emb = self.pbs_embedding(torch.clamp(features['PBS_len'], max=49))
        rt_emb = self.rt_embedding(torch.clamp(features['RT_len'], max=49))
        edit_pos_emb = self.edit_pos_embedding(torch.clamp(features['edit_pos'], max=49))
        edit_len_emb = self.edit_len_embedding(torch.clamp(features['edit_len'], max=19))
        rha_len_emb = self.rha_len_embedding(torch.clamp(features['RHA_len'], max=49))

        # Determine edit type: stack type_sub, type_ins, type_del and take argmax along dimension 1
        edit_type = torch.argmax(torch.stack([
            features['type_sub'],
            features['type_ins'],
            features['type_del']
        ], dim=1), dim=1)
        edit_type_emb = self.edit_type_embedding(edit_type)

        # Process continuous numerical features
        tm_features = self.tm_features_linear(features['tm_features']).float()
        mfe_features = self.mfe_features_linear(features['mfe_features']).float()
        gc_features = self.gc_linear(features['GC_content'].unsqueeze(1)).float()

        # Concatenate all non-sequence features
        combined = torch.cat([
            pbs_emb, rt_emb, edit_pos_emb, edit_len_emb, rha_len_emb,
            edit_type_emb, tm_features, mfe_features, gc_features
        ], dim=1)  # shape: [batch, 80]

        # Transform to fixed feature dimension
        feature_vector = self.feature_linear(combined)  # shape: [batch, feature_dim]

        # Concatenate sequence features with processed non-sequence features
        final_features = torch.cat([seq_features, feature_vector.float()], dim=1)  # shape: [batch, seq_dim + feature_dim]

        # Pass through final prediction layers
        output = self.combined_linear(final_features)

        # Scale the output to a percentage (0-100)
        return self.sigmoid(output) * 100


In [57]:
model = CRISPRPredictor(tokenizer).to(device).to(torch.float32)


In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import time

def train_crispr_model(train_loader, val_loader, tokenizer, device='cuda', epochs=50):
    # Initialize the model and move it to the specified device
    model = CRISPRPredictor(tokenizer).to(device)

    # Set up optimizer with weight decay for regularization
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

    # Set up a learning rate scheduler that reduces LR when the validation loss plateaus
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    # Mean Squared Error loss for regression
    criterion = nn.MSELoss()

    best_val_loss = float('inf')
    best_model_state = None
    train_losses = []
    val_losses = []
    val_r2_scores = []

    print("Starting training...")
    for epoch in range(epochs):
        start_time = time.time()

        # Training phase
        model.train()
        train_loss = 0
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            # Move BERT tokenized inputs to device
            wt_tokens = {k: v.to(device).long() for k, v in inputs['wt_tokens'].items()}
            edited_tokens = {k: v.to(device).long() for k, v in inputs['edited_tokens'].items()}

            # Move numerical features to device (our integrated dataset returns these under 'numerical_features')
            features = {}
            for k, v in inputs['numerical_features'].items():
                if isinstance(v, torch.Tensor):
                    features[k] = v.to(device).float()
                else:
                    features[k] = torch.tensor(v, device=device, dtype=torch.float32)

            batch_inputs = {
                'wt_tokens': wt_tokens,
                'edited_tokens': edited_tokens,
                'numerical_features': features
            }
            targets = targets.to(device).float()

            optimizer.zero_grad()
            outputs = model(batch_inputs).float()
            loss = criterion(outputs.squeeze(), targets)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            if batch_idx % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation phase
        model.eval()
        val_loss = 0
        all_targets = []
        all_predictions = []
        with torch.no_grad():
            for inputs, targets in val_loader:
                wt_tokens = {k: v.to(device) for k, v in inputs['wt_tokens'].items()}
                edited_tokens = {k: v.to(device) for k, v in inputs['edited_tokens'].items()}
                features = {}
                for k, v in inputs['numerical_features'].items():
                    if isinstance(v, torch.Tensor):
                        features[k] = v.to(device).float()
                    else:
                        features[k] = torch.tensor(v, device=device, dtype=torch.float32)

                batch_inputs = {
                    'wt_tokens': wt_tokens,
                    'edited_tokens': edited_tokens,
                    'numerical_features': features
                }
                targets = targets.to(device).float()
                outputs = model(batch_inputs).float()
                loss = criterion(outputs.squeeze(), targets)
                val_loss += loss.item()

                all_targets.extend(targets.cpu().numpy())
                all_predictions.extend(outputs.squeeze().cpu().numpy())

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        r2 = r2_score(all_targets, all_predictions)
        val_r2_scores.append(r2)
        rmse = np.sqrt(mean_squared_error(all_targets, all_predictions))

        scheduler.step(val_loss)

        # Save the best model so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict().copy()
            torch.save({
                'epoch': epoch,
                'model_state_dict': best_model_state,
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': best_val_loss,
                'r2_score': r2,
            }, 'best_crispr_model.pt')

        time_taken = time.time() - start_time
        print(f"Epoch {epoch+1}/{epochs} completed in {time_taken:.2f}s")
        print(f"Training Loss: {train_loss:.4f} | Validation Loss: {val_loss:.4f}")
        print(f"R² Score: {r2:.4f} | RMSE: {rmse:.4f}")
        print(f"Current LR: {optimizer.param_groups[0]['lr']:.6f}")
        print("-" * 50)

    # Load the best model state for final evaluation
    model.load_state_dict(best_model_state)

    # Plot training curves
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curves')

    plt.subplot(1, 2, 2)
    plt.plot(val_r2_scores, label='R² Score')
    plt.xlabel('Epoch')
    plt.ylabel('R² Score')
    plt.legend()
    plt.title('R² Score')

    plt.tight_layout()
    plt.savefig('training_history.png')

    return model, best_val_loss, val_r2_scores[-1]

# ---------------------------
# Main block to run training
# ---------------------------
if __name__ == "__main__":
    # Set device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Prepare data. Make sure to use the integrated data preparation function.
    # Replace 'data/DeepPrime_dataset_final_Feat8.csv' with your actual CSV file path.
    train_loader, val_loader, tokenizer = prepare_integrated_crispr_data(
        'DT.csv',
        batch_size=32,
        use_bert=True,
        use_onehot=True
    )

    # Train the model
    model, best_loss, r2 = train_crispr_model(train_loader, val_loader, tokenizer, device=device, epochs=50)

    print(f"Training completed with best validation loss: {best_loss:.4f} and R² score: {r2:.4f}")


Using device: cpu
Preprocessing one-hot encodings...


100%|██████████| 231034/231034 [03:35<00:00, 1072.87it/s]


Preprocessing one-hot encodings...


100%|██████████| 57759/57759 [01:03<00:00, 911.39it/s] 
C:\Users\chidr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting training...


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)